# Fine Tuning MarianMT Model

In [1]:
import pandas as pd 
import numpy as np 
import torch
from torch.utils.data import TensorDataset, DataLoader
import tqdm as tqdm
from evaluate import load
from transformers import MarianMTModel, MarianTokenizer

/opt/anaconda3/envs/NLP_25/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook we will walk through fine tuning the pre-existing MarianMT english to spanish model using the OpenSubtitles english and spanish text.

## Preprocessing Data

First we will parse the data and a sentence in a row, with each column representing the spanish version and english version. We will then split the data appropriately. We will export as a csv for later usage

### Initial Importing of Data

In [ ]:
# Grabbing English sentences
with open('../data/en-es/OpenSubtitles.en-es.en') as en_text:
    english_sent = [line.strip() for line in en_text]

In [ ]:
with open('../data/en-es/OpenSubtitles.en-es.es') as es_text:
    spanish_sent = [line.strip() for line in es_text]

In [ ]:
# Assert they are the same size
len(english_sent), len(spanish_sent)

In [ ]:
sentences = pd.DataFrame({
    'english': english_sent,
    'spanish': spanish_sent
})

In [ ]:
sentences.to_csv('en-es_Full_Dataset.csv')

In [ ]:
sentences

### Importing Data from csv file

In [2]:
sentences = pd.read_csv('../data/en-es_Full_Dataset.csv')

### Splitting Data

We will split the data into 3 different sets. A test, validation, and test set. Each set will have english and spanish sentences. 

We will reserve 70% of our data for training, 10% for validation, and 20% for testing. 

In [3]:
sentences_sample = sentences[:int(len(sentences)*.000001)]
len(sentences_sample)

105

In [ ]:
# Preparing Training Data
train_X = sentences['english'][:int(len(sentences) * .7)] # English only for input
train_y = sentences['spanish'][:int(len(sentences) * .7)] # Spanish for target

# Preparing Testing Data
test_X = sentences['english'][int(len(sentences) * .7) : int(len(sentences) * .9)]
test_y = sentences['spanish'][int(len(sentences) * .7) : int(len(sentences) * .9)]

# Preparing Validation Data
val_X = sentences['english'][int(len(sentences) * .9) : int(len(sentences))]
val_y = sentences['spanish'][int(len(sentences) * .9) : int(len(sentences))]


In [4]:
# Creating sample size to test code

# Preparing Training Data
train_X = sentences_sample['english'][:int(len(sentences_sample) * .7)] # English only for input
train_y = sentences_sample['spanish'][:int(len(sentences_sample) * .7)] # Spanish for target

# Preparing Testing Data
test_X = sentences_sample['english'][int(len(sentences_sample) * .7) : int(len(sentences_sample) * .9)]
test_y = sentences_sample['spanish'][int(len(sentences_sample) * .7) : int(len(sentences_sample) * .9)]

# Preparing Validation Data
val_X = sentences_sample['english'][int(len(sentences_sample) * .9) : int(len(sentences_sample))]
val_y = sentences_sample['spanish'][int(len(sentences_sample) * .9) : int(len(sentences_sample))]

### Preparing Data

Here we are converting our data into a datatype that is acceptable for the DataLoader class. The DataLoader class loads a certain amount of data to input to the model based on the batch size during training.

First we need to tokenize our inputs so we can represent our text as a numerical value

In [5]:
model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)

/opt/anaconda3/envs/NLP_25/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


This class extends the Dataset class. What it does is returns the tokenized english sentence and spanish sentence for model inputs.

In [87]:
class MultiLabelDataset(torch.utils.data.Dataset):
    def __init__(self, text, labels, tokenizer):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        
    def __len__(self):
        return len(self.text)
        
    def __getitem__(self, index):
        text = self.text[index]
        target = self.targets[index]
        
        # Tokenize source text
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128 , # Adjust as needed
            pad_to_max_length=True,
            return_tensors=None
        )
        
        # Tokenize target text
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                target,
                truncation=True,
                max_length=128, # Adjust as needed
                pad_to_max_length=True,
                return_tensors=None
            )
        
        # Remove batch dimension created by return_tensors="pt"
        source = inputs['input_ids']
        targets = labels['input_ids']
        
        return {
            'ids': torch.tensor(source, dtype=torch.long),
            'targets': torch.tensor(targets, dtype=torch.long)
        }

In [88]:
train_inputs = tokenizer.encode_plus(train_X[0], truncation=True)


In [89]:
# Converting our data into a torch tensor
train_X = np.array(train_X)
train_y = np.array(train_y)

test_X = np.array(test_X)
test_y = np.array(test_y)

val_X = np.array(val_X)
val_y = np.array(val_y)

Creating a dataset based on our sentences

In [90]:
train_data = MultiLabelDataset(train_X, train_y, tokenizer)
test_data = MultiLabelDataset(test_X, test_y, tokenizer)
val_data = MultiLabelDataset(val_X, val_y, tokenizer)

In [91]:
batch_size = 16

# Create DataLoaders 
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

## Fine-Tuning

We will be fine-tuning the MarianMT with our data

In [92]:
model_name = 'Helsinki-NLP/opus-mt-en-es'
model = MarianMTModel.from_pretrained(model_name)
base_model = MarianMTModel.from_pretrained(model_name)

In [93]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print (x)
else:
    print ("MPS device not found.")
model.to(device)    
base_model.to(device)

tensor([1.], device='mps:0')


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [98]:
def train(model, training_loader, optimizer):
    model.train()
    for data in training_loader:
        ids = data['ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        optimizer.zero_grad()
        outputs = model(input_ids=ids, decoder_input_ids=targets)
        optimizer.step()

In [99]:
def val(model, val_loader, tokenizer, bertscore):
    model.eval()
    all_predictions = []
    all_references = []
    
    with torch.no_grad():
        for data in tqdm.tqdm(val_loader):
            targets = data['targets'].to(device, dtype = torch.long)
            ids = data['ids'].to(device, dtype = torch.long)
            generated_ids = model.generate(input_ids = ids)
            
            predictions = tokenizer.batch_decode(generated_ids, kip_special_tokens=True)[0]
            references = tokenizer.batch_decode(targets, skip_special_tokens=True)[0]
            
            all_predictions.append(predictions)
            all_references.append(references)
        results = bertscore.compute(predictions=all_predictions, references=all_references, lang='es')
    return results['f1']
        

In [100]:
def test(model, test_loader, tokenizer, bertscore):
    model.eval()
    all_predictions = []
    all_references = []
    
    with torch.no_grad():
        for data in tqdm.tqdm(test_loader):
            targets = data['targets'].to(device, dtype = torch.long)
            ids = data['ids'].to(device, dtype = torch.long)
            generated_ids = model.generate(input_ids = ids)
            
            predictions = tokenizer.batch_decode(generated_ids, kip_special_tokens=True)[0]
            references = tokenizer.batch_decode(targets, skip_special_tokens=True)[0]
            
            all_predictions.append(predictions)
            all_references.append(references)
        results = bertscore.compute(predictions=all_predictions, references=all_references, lang='es')
    return results['f1'], all_predictions, all_references

In [101]:
optim = torch.optim.AdamW(model.parameters(), lr=.00002)
NUM_EPOCH = 3
bertscore = load("bertscore")

for epoch in range(NUM_EPOCH):
    train(model, train_loader, optim)
    f1_score = val(model, val_loader, tokenizer, bertscore)
    print(f'Epoch: {epoch+1}, BERTScore F1: {f1_score}')

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

## Evaluate

Here we will evalute our model. We will compare our fine tuned model to our base model using the BERTScore which finds the cosine similarity between two translated words.

In [ ]:
f1_score_based_model, pred, ref = test(base_model, test_loader, tokenizer, bertscore)

In [ ]:
np.mean(f1_score_based_model)

In [ ]:
pred

In [ ]:
ref